# Actividad 2 – Machine Learning II  
## Modelos no lineales para Churn: Árbol de Decisión y Random Forest

**Contexto:** en la Actividad 1 se trabajó churn con modelos lineales (Regresión Logística).  
En esta Actividad 2 se extiende el análisis a modelos basados en árboles, con foco en:

- Búsqueda sistemática de hiperparámetros con GridSearchCV y RandomizedSearchCV.
- Comparación de desempeño y costo computacional.
- Análisis de varianza/estabilidad de predicciones en Random Forest al variar el número de árboles.
- Evaluación con métricas y curvas adecuadas para desbalance de clases (ROC-AUC, PR-AUC, F1).

Se mantiene el mismo preprocesamiento
 usado en la Actividad 1 (imputación + one-hot + escalamiento).


In [ ]:
# =========================
# 0) Librerías
# =========================
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import time
from pprint import pprint

from sklearn.model_selection import StratifiedKFold, GridSearchCV, RandomizedSearchCV
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.impute import SimpleImputer

from sklearn.tree import DecisionTreeClassifier, plot_tree
from sklearn.ensemble import RandomForestClassifier

from sklearn.metrics import (
    confusion_matrix,
    classification_report,
    accuracy_score,
    precision_score,
    recall_score,
    f1_score,
    roc_curve,
    auc,
    precision_recall_curve,
    average_precision_score
)

import warnings
warnings.filterwarnings("ignore")

RANDOM_STATE = 42
np.random.seed(RANDOM_STATE)


## Primer Paso. Dataset y variables

Se utiliza el dataset data-churn.csv (mismo del ejercicio anterior).  
La variable objetivo Churn se transforma a binaria: **Yes → 1**, **No → 0**.


In [ ]:
# =========================
# 1) Carga del dataset
# =========================
df = pd.read_csv("data-churn.csv")
df.head()


In [ ]:
# Variable objetivo binaria
df["Churn"] = df["Churn"].map({"Yes": 1, "No": 0})

print("Tamaño del dataset:", df.shape)
print("\nProporción de clases (Churn):")
display(df["Churn"].value_counts(normalize=True).rename("proporción"))


In [ ]:
# Revisión rápida de faltantes
display(df.isna().sum().sort_values(ascending=False).head(15))


## Paso 1. Preprocesamiento (con el enfoque de la Actividad 1)

Decisiones principales:
- **Faltantes:** imputación simple (mediana en numéricas, más frecuente en categóricas).
- **Categóricas:** One-Hot Encoding con drop='first' para evitar multicolinealidad perfecta.
- **Numéricas:** estandarización con StandardScaler.

Aunque los árboles no requieren escalamiento, se mantiene el mismo preprocesamiento por consistencia con la Actividad 1.


In [ ]:
# =========================
# 2) Features y target
# =========================
target = "Churn"

X = df.drop(columns=[target]).copy()
y = df[target].copy()

# Forzar numéricas a tipo numérico (por ejemplo TotalCharges puede venir como texto)
for col in X.columns:
    if X[col].dtype == "object":
        # Intentar convertir a numérico si aplica; si no, queda categórica.
        pass

numeric_features = X.select_dtypes(include=[np.number]).columns.tolist()
categorical_features = X.select_dtypes(exclude=[np.number]).columns.tolist()

print("Numéricas:", numeric_features)
print("Categóricas:", categorical_features)


In [ ]:
# =========================
# 3) Preprocesador (Actividad 1)
# =========================
numeric_transformer = Pipeline(steps=[
    ("imputer", SimpleImputer(strategy="median")),
    ("scaler", StandardScaler())
])

categorical_transformer = Pipeline(steps=[
    ("imputer", SimpleImputer(strategy="most_frequent")),
    ("onehot", OneHotEncoder(drop="first", handle_unknown="ignore"))
])

preprocessor = ColumnTransformer(
    transformers=[
        ("num", numeric_transformer, numeric_features),
        ("cat", categorical_transformer, categorical_features)
    ],
    remainder="drop"
)

cv5 = StratifiedKFold(n_splits=5, shuffle=True, random_state=RANDOM_STATE)


## Funciones auxiliares
Se definen funciones para:
- Obtener nombres de features tras el preprocesamiento.
- Evaluar modelos con k-fold estratificado (métricas + curvas ROC/PR promedio).


In [ ]:
def get_feature_names_from_preprocessor(preprocessor: ColumnTransformer):
    """Devuelve nombres de columnas luego del ColumnTransformer."""
    feature_names = []
    for name, transformer, cols in preprocessor.transformers_:
        if name == "remainder" and transformer == "drop":
            continue
        if hasattr(transformer, "named_steps"):
            # Pipeline
            last_step = list(transformer.named_steps.values())[-1]
            if hasattr(last_step, "get_feature_names_out"):
                try:
                    names = last_step.get_feature_names_out(cols)
                except:
                    names = last_step.get_feature_names_out()
                feature_names.extend(names)
            else:
                feature_names.extend(cols)
        else:
            if hasattr(transformer, "get_feature_names_out"):
                try:
                    names = transformer.get_feature_names_out(cols)
                except:
                    names = transformer.get_feature_names_out()
                feature_names.extend(names)
            else:
                feature_names.extend(cols)
    return [str(n) for n in feature_names]


def evaluate_model_cv(model, X, y, cv, title="Modelo", plot_curves=True, verbose=True):
    """Evalúa un modelo con StratifiedKFold: métricas por fold + resumen + curvas promedio."""
    per_fold = []
    roc_curves = []
    pr_curves = []

    mean_fpr = np.linspace(0, 1, 200)
    mean_recall = np.linspace(0, 1, 200)

    y_true_all, y_pred_all = [], []
    # (Para PR-AUC usamos average_precision_score, que trabaja con probabilidades)
    y_proba_all = []

    for fold, (tr, te) in enumerate(cv.split(X, y), start=1):
        X_tr, X_te = X.iloc[tr], X.iloc[te]
        y_tr, y_te = y.iloc[tr], y.iloc[te]

        model.fit(X_tr, y_tr)
        y_pred = model.predict(X_te)
        y_proba = model.predict_proba(X_te)[:, 1]

        acc = accuracy_score(y_te, y_pred)
        prec = precision_score(y_te, y_pred, zero_division=0)
        rec = recall_score(y_te, y_pred, zero_division=0)
        f1 = f1_score(y_te, y_pred, zero_division=0)
        pr_auc = average_precision_score(y_te, y_proba)

        fpr, tpr, _ = roc_curve(y_te, y_proba)
        roc_auc = auc(fpr, tpr)

        per_fold.append({
            "fold": fold,
            "accuracy": acc,
            "precision": prec,
            "recall": rec,
            "f1": f1,
            "roc_auc": roc_auc,
            "pr_auc": pr_auc
        })

        # Curvas interpoladas (promedio)
        tpr_i = np.interp(mean_fpr, fpr, tpr)
        tpr_i[0] = 0.0
        roc_curves.append(tpr_i)

        p, r, _ = precision_recall_curve(y_te, y_proba)
        order = np.argsort(r)
        r_sorted = r[order]
        p_sorted = p[order]
        p_i = np.interp(mean_recall, r_sorted, p_sorted)
        pr_curves.append(p_i)

        y_true_all.extend(y_te)
        y_pred_all.extend(y_pred)
        y_proba_all.extend(y_proba)

    df_fold = pd.DataFrame(per_fold)
    summary = pd.DataFrame({
        "mean": df_fold.drop(columns=["fold"]).mean(),
        "std": df_fold.drop(columns=["fold"]).std()
    })

    if verbose:
        print(f"\n=== {title} ===")
        display(df_fold)
        display(summary)

        cm = confusion_matrix(y_true_all, y_pred_all)
        print("Matriz de confusión global (todos los folds):")
        print(cm)
        print("\nReporte de clasificación global:")
        print(classification_report(y_true_all, y_pred_all, digits=3))

    curves = None
    if plot_curves:
        mean_tpr = np.mean(roc_curves, axis=0)
        mean_tpr[-1] = 1.0
        mean_roc_auc = auc(mean_fpr, mean_tpr)

        mean_precision = np.mean(pr_curves, axis=0)
        mean_pr_auc = auc(mean_recall, mean_precision)

        plt.figure()
        plt.plot(mean_fpr, mean_tpr, label=f"ROC promedio (AUC≈{mean_roc_auc:.3f})")
        plt.plot([0, 1], [0, 1], linestyle="--")
        plt.xlabel("False Positive Rate")
        plt.ylabel("True Positive Rate (Recall)")
        plt.title(f"Curva ROC promedio – {title}")
        plt.legend()
        plt.show()

        plt.figure()
        plt.plot(mean_recall, mean_precision, label=f"PR promedio (AUC≈{mean_pr_auc:.3f})")
        plt.xlabel("Recall")
        plt.ylabel("Precision")
        plt.title(f"Curva Precision–Recall promedio – {title}")
        plt.legend()
        plt.show()

        curves = {
            "mean_fpr": mean_fpr,
            "mean_tpr": mean_tpr,
            "mean_recall": mean_recall,
            "mean_precision": mean_precision
        }

    return df_fold, summary, curves


# Segundo paso. Árbol de decisión (Grid Search vs Random Search)

Se ajusta un DecisionTreeClassifier utilizando el mismo preprocesamiento de la Actividad 1.

## 1.1 Grilla de hiperparámetros y justificación
- **max_depth:** controla la profundidad del árbol. Valores altos aumentan riesgo de sobreajuste.
- **min_samples_split / min_samples_leaf:** evitan particiones demasiado específicas.
- **criterion:** cambia la forma de medir la “calidad” de una división (gini/entropy/log_loss).

Se utiliza como métrica principal PR-AUC (Average Precision), adecuada cuando `churn=1` es minoritario.


In [ ]:
# =========================
# 4) Árbol de decisión + búsqueda de hiperparámetros
# =========================
pipe_dt = Pipeline(steps=[
    ("preprocess", preprocessor),
    ("clf", DecisionTreeClassifier(random_state=RANDOM_STATE))
])

param_grid_dt = {
    "clf__max_depth": [None, 3, 5, 8, 12, 16],
    "clf__min_samples_split": [2, 10, 25, 50],
    "clf__min_samples_leaf": [1, 5, 10, 25],
    "clf__criterion": ["gini", "entropy", "log_loss"]
}

scoring = "average_precision"  # PR-AUC


In [ ]:
# 1) Grid Search CV
grid_dt = GridSearchCV(
    estimator=pipe_dt,
    param_grid=param_grid_dt,
    cv=cv5,
    scoring=scoring,
    n_jobs=-1,
    refit=True,
    return_train_score=False
)

t0 = time.perf_counter()
grid_dt.fit(X, y)
grid_time = time.perf_counter() - t0

print("GridSearchCV time (s):", round(grid_time, 3))
print("Best params (Grid):")
pprint(grid_dt.best_params_)
print("Best score (Grid) [PR-AUC]:", grid_dt.best_score_)


In [ ]:
# 2) Random Search CV
rand_dt = RandomizedSearchCV(
    estimator=pipe_dt,
    param_distributions=param_grid_dt,
    n_iter=50,                   # balance entre cobertura y tiempo
    cv=cv5,
    scoring=scoring,
    n_jobs=-1,
    random_state=RANDOM_STATE,
    refit=True,
    return_train_score=False
)

t0 = time.perf_counter()
rand_dt.fit(X, y)
rand_time = time.perf_counter() - t0

print("RandomizedSearchCV time (s):", round(rand_time, 3))
print("Best params (Random):")
pprint(rand_dt.best_params_)
print("Best score (Random) [PR-AUC]:", rand_dt.best_score_)


In [ ]:
# Comparación directa: tiempo y mejor score
comparison_search = pd.DataFrame({
    "method": ["GridSearchCV", "RandomizedSearchCV"],
    "time_seconds": [grid_time, rand_time],
    "best_pr_auc": [grid_dt.best_score_, rand_dt.best_score_]
})

display(comparison_search)


In [ ]:
# Selección del mejor árbol (por PR-AUC; en empate, el más rápido)
if rand_dt.best_score_ > grid_dt.best_score_:
    best_dt_search = rand_dt
elif grid_dt.best_score_ > rand_dt.best_score_:
    best_dt_search = grid_dt
else:
    # Empate: elegir el método más rápido
    best_dt_search = rand_dt if rand_time < grid_time else grid_dt

best_dt = best_dt_search.best_estimator_

print("Método elegido:", type(best_dt_search).__name__)
print("Mejores hiperparámetros:")
pprint(best_dt_search.best_params_)


## 1.2 Evaluación del mejor Árbol (k-fold)

Se evalúa el árbol seleccionado con k-fold estratificado reportando métricas y curvas ROC/PR.


In [ ]:
df_dt, summary_dt, curves_dt = evaluate_model_cv(
    best_dt, X, y, cv=cv5, title="Decision Tree óptimo", plot_curves=True, verbose=True
)


# Tercer paso. Visualización e interpretación del árbol óptimo

Se entrena el árbol óptimo en todo el dataset para visualizar sus primeras divisiones.
Para mejorar legibilidad, se limita la profundidad visualizada.


In [ ]:
# Entrenar el mejor árbol en todo el dataset para visualizarlo
best_dt.fit(X, y)

dt_clf = best_dt.named_steps["clf"]
feature_names = get_feature_names_from_preprocessor(best_dt.named_steps["preprocess"])

print("Total features tras preprocesamiento:", len(feature_names))
print("Ejemplos:", feature_names[:10])


In [ ]:
plt.figure(figsize=(18, 10))
plot_tree(
    dt_clf,
    feature_names=feature_names,
    class_names=["No churn (0)", "Churn (1)"],
    filled=True,
    max_depth=2,            # ajustar si necesitas más/menos detalle
    impurity=False,
    proportion=True,
    rounded=True,
    fontsize=8
)
plt.title("Árbol de decisión óptimo (profundidad visual limitada)")
plt.show()


**Interpretación (guía breve):**
- Las variables que aparecen en los **primeros nodos** suelen ser las más influyentes en la decisión del árbol.
- A diferencia de la regresión logística (coeficientes globales), el árbol ofrece reglas tipo *“si… entonces…”*,
  lo que mejora la interpretabilidad a nivel de reglas, pero puede ser inestable si el árbol es muy profundo.


# Cuarto paso. Random Forest y análisis de varianza (número de árboles)

Se estudia la estabilidad de un Random Forest al variar el número de árboles:

n_estimators = [2, 4, 8, 16, 32, 64, 128]

Para cada configuración:
- Se entrena con k-fold estratificado.
- Se registra el desempeño (**F1, ROC-AUC, PR-AUC**).
- Se estima la **varianza de las probabilidades predichas entre folds** de esta forma:
  1) para cada fold, se entrena un modelo con el set de entrenamiento del fold;  
  2) ese modelo predice probabilidades sobre **todo X**;  
  3) se calcula la varianza por muestra entre los modelos de folds y se reporta el promedio (estabilidad global).

Esto refleja cuán sensibles son las predicciones a cambios en los datos de entrenamiento.


In [ ]:
def rf_variance_study(X, y, cv, n_estimators_list, rf_params=None):
    if rf_params is None:
        rf_params = {}

    rows = []
    for n_estimators in n_estimators_list:
        fold_models_proba_full = []  # proba sobre todo X por cada fold-model
        fold_metrics = []
        total_fit_time = 0.0

        for tr, te in cv.split(X, y):
            X_tr, X_te = X.iloc[tr], X.iloc[te]
            y_tr, y_te = y.iloc[tr], y.iloc[te]

            model = Pipeline(steps=[
                ("preprocess", preprocessor),
                ("clf", RandomForestClassifier(
                    n_estimators=n_estimators,
                    random_state=RANDOM_STATE,
                    n_jobs=-1,
                    **rf_params
                ))
            ])

            t0 = time.perf_counter()
            model.fit(X_tr, y_tr)
            total_fit_time += (time.perf_counter() - t0)

            # Predicciones en fold (métricas)
            y_proba = model.predict_proba(X_te)[:, 1]
            y_pred = (y_proba >= 0.5).astype(int)

            f1 = f1_score(y_te, y_pred, zero_division=0)
            pr_auc = average_precision_score(y_te, y_proba)

            fpr, tpr, _ = roc_curve(y_te, y_proba)
            roc_auc = auc(fpr, tpr)

            fold_metrics.append((f1, roc_auc, pr_auc))

            # Probabilidades sobre todo X (para varianza entre folds)
            proba_full = model.predict_proba(X)[:, 1]
            fold_models_proba_full.append(proba_full)

        fold_models_proba_full = np.vstack(fold_models_proba_full)  # shape: (n_folds, n_samples)
        var_per_sample = np.var(fold_models_proba_full, axis=0)     # varianza entre folds por muestra
        mean_var = float(np.mean(var_per_sample))

        fold_metrics = np.array(fold_metrics)  # (n_folds, 3)
        rows.append({
            "n_estimators": n_estimators,
            "mean_variance_proba": mean_var,
            "f1_mean": fold_metrics[:, 0].mean(),
            "roc_auc_mean": fold_metrics[:, 1].mean(),
            "pr_auc_mean": fold_metrics[:, 2].mean(),
            "fit_time_total_seconds": total_fit_time
        })

    return pd.DataFrame(rows)


n_estimators_list = [2, 4, 8, 16, 32, 64, 128]
df_var = rf_variance_study(X, y, cv=cv5, n_estimators_list=n_estimators_list, rf_params={"max_features": "sqrt"})
display(df_var)


In [ ]:
# Gráfico: varianza vs número de árboles
plt.figure()
plt.plot(df_var["n_estimators"], df_var["mean_variance_proba"], marker="o")
plt.xscale("log", base=2)
plt.xlabel("Número de árboles (n_estimators)")
plt.ylabel("Varianza promedio de probabilidades (entre folds)")
plt.title("Estabilidad de predicciones: Varianza vs número de árboles")
plt.show()


In [ ]:
# Gráfico: métricas vs número de árboles
plt.figure()
plt.plot(df_var["n_estimators"], df_var["f1_mean"], marker="o", label="F1")
plt.plot(df_var["n_estimators"], df_var["roc_auc_mean"], marker="o", label="ROC-AUC")
plt.plot(df_var["n_estimators"], df_var["pr_auc_mean"], marker="o", label="PR-AUC")
plt.xscale("log", base=2)
plt.xlabel("Número de árboles (n_estimators)")
plt.ylabel("Métrica promedio (k-fold)")
plt.title("Desempeño vs número de árboles")
plt.legend()
plt.show()


**Discusión esperada (guía):**
- Al aumentar n_estimators, es común observar reducción de varianza (mayor estabilidad).
- El desempeño puede mejorar hasta cierto punto y luego estabilizarse.
- El costo computacional aumenta (tiempo total de entrenamiento).


# Quinto paso. Selección del mejor Random Forest (búsqueda de hiperparámetros)

Se define una grilla/distribuciones de hiperparámetros y se utiliza RandomizedSearchCV
para equilibrar cobertura y tiempo computacional.  
La métrica principal sigue siendo PR-AUC (Average Precision).


In [ ]:
pipe_rf = Pipeline(steps=[
    ("preprocess", preprocessor),
    ("clf", RandomForestClassifier(random_state=RANDOM_STATE, n_jobs=-1))
])

param_dist_rf = {
    "clf__n_estimators": [64, 128, 256],
    "clf__max_depth": [None, 5, 10, 15],
    "clf__min_samples_leaf": [1, 5, 10],
    "clf__max_features": ["sqrt", "log2", None],
    "clf__class_weight": [None, "balanced"]
}

rand_rf = RandomizedSearchCV(
    estimator=pipe_rf,
    param_distributions=param_dist_rf,
    n_iter=40,
    cv=cv5,
    scoring="average_precision",
    n_jobs=-1,
    random_state=RANDOM_STATE,
    refit=True,
    return_train_score=False
)

t0 = time.perf_counter()
rand_rf.fit(X, y)
rf_search_time = time.perf_counter() - t0

print("RandomizedSearchCV RF time (s):", round(rf_search_time, 3))
print("Best params (RF):")
pprint(rand_rf.best_params_)
print("Best score (RF) [PR-AUC]:", rand_rf.best_score_)

best_rf = rand_rf.best_estimator_


## 4.1 Evaluación del mejor Random Forest (k-fold)
Se evalúa el Random Forest seleccionado, reportando métricas y curvas ROC/PR.


In [ ]:
df_rf, summary_rf, curves_rf = evaluate_model_cv(
    best_rf, X, y, cv=cv5, title="Random Forest óptimo", plot_curves=True, verbose=True
)


# Sexto paso. Comparación final y mejora con pesos por clase

Según la pauta:
- Se intenta mejorar usando class_weight="balanced".
- Se comparan Árbol óptimo vs Random Forest óptimo.
- Se reportan métricas (Accuracy, Precision, Recall, F1) y curvas ROC/PR.


In [ ]:
# Mejorar Árbol con class_weight (si no estaba ya considerado)
dt_balanced = Pipeline(steps=[
    ("preprocess", preprocessor),
    ("clf", DecisionTreeClassifier(
        random_state=RANDOM_STATE,
        class_weight="balanced",
        **{k.replace("clf__", ""): v for k, v in best_dt_search.best_params_.items()}
    ))
])

df_dt_b, summary_dt_b, _ = evaluate_model_cv(
    dt_balanced, X, y, cv=cv5, title="Decision Tree (class_weight='balanced')", plot_curves=False, verbose=False
)

# Mejorar RF con class_weight (si no estaba ya)
best_rf_params = best_rf.named_steps["clf"].get_params()
best_rf_params.update({"class_weight": "balanced"})

rf_balanced = Pipeline(steps=[
    ("preprocess", preprocessor),
    ("clf", RandomForestClassifier(**best_rf_params))
])

df_rf_b, summary_rf_b, _ = evaluate_model_cv(
    rf_balanced, X, y, cv=cv5, title="Random Forest (class_weight='balanced')", plot_curves=False, verbose=False
)

display(summary_dt_b)
display(summary_rf_b)


In [ ]:
# Tabla comparativa final (promedios)
def summarize_for_compare(summary, model_name):
    out = summary["mean"].copy()
    out.name = model_name
    return out

compare = pd.concat([
    summarize_for_compare(summary_dt, "Decision Tree"),
    summarize_for_compare(summary_rf, "Random Forest"),
    summarize_for_compare(summary_dt_b, "Decision Tree (balanced)"),
    summarize_for_compare(summary_rf_b, "Random Forest (balanced)"),
], axis=1)

display(compare)


# Séptimo paso. Respuestas a preguntas de la pauta

**Relación entre varianza, ensambles y capacidad de generalización**

Random Forest se basa en la combinación de múltiples árboles de decisión entrenados de forma aleatoria, tanto en la selección de observaciones (bootstrap) como en el subconjunto de variables utilizadas en cada división. Este enfoque permite reducir la varianza respecto a un árbol individual, haciendo que el modelo sea menos sensible a cambios en los datos de entrenamiento y, en consecuencia, mejore su estabilidad y capacidad de generalización.

**Ventajas y desventajas de árboles de decisión frente a modelos lineales**

Los árboles de decisión tienen la ventaja de capturar relaciones no lineales y generar reglas del tipo “si–entonces”, lo que facilita su interpretación, especialmente cuando el árbol es poco profundo. Sin embargo, pueden ser propensos al sobreajuste.
En contraste, los modelos lineales, como la regresión logística, son conceptualmente más simples y ofrecen una interpretación global más directa a través de sus coeficientes, aunque pueden resultar limitados cuando las relaciones entre variables y la respuesta son complejas o no lineales.

**¿Cuándo preferir un árbol interpretable en lugar de un Random Forest?**

Un árbol de decisión interpretable puede ser preferible cuando la transparencia del modelo es prioritaria, por ejemplo, en contextos de auditoría, cumplimiento normativo o comunicación con áreas no técnicas del negocio. En estos casos, se acepta un posible sacrificio en desempeño predictivo a cambio de una explicación clara y comprensible de las decisiones del modelo.

**Conexión con la teoría: reducción de varianza mediante ensambles**

Desde un punto de vista teórico, los métodos de ensamble reducen la varianza al promediar las predicciones de múltiples modelos individuales, lo que atenúa errores específicos de cada uno. Esta idea se refleja en los resultados empíricos obtenidos, donde se observa una disminución de la varianza de las predicciones a medida que aumenta el número de árboles en el Random Forest.

**Métrica prioritaria desde la perspectiva del negocio (retención de clientes)**

En campañas de retención, resulta especialmente relevante minimizar la cantidad de clientes que abandonan el servicio sin ser detectados por el modelo. Por este motivo, métricas como Recall, así como métricas orientadas a la clase positiva como PR-AUC y F1, suelen ser más adecuadas que la accuracy. No obstante, la selección final debe considerar la capacidad operativa del equipo de retención, evitando generar un volumen excesivo de falsos positivos.


## Conclusiones
- Se seleccionó un árbol de decisión mediante Grid/Random Search y se compararon tiempos y desempeño.
- Se visualizó el árbol óptimo y se discutió su interpretabilidad.
- En Random Forest, al aumentar el número de árboles se observó una tendencia a mayor estabilidad (menor varianza),
  con un costo computacional mayor.
- La comparación final sugiere que Random Forest suele superar a un árbol individual por su capacidad de generalización,
  especialmente en problemas con ruido y desbalance como churn.
